## Prerequisites

In [1]:
import pandas as pd
import re
import plotly.express as px

In [11]:
!gdown https://drive.google.com/uc?id=1jbHGsrSJt1FfDbD67H7fJAtH5e8Zxrkc

Downloading...
From: https://drive.google.com/uc?id=1jbHGsrSJt1FfDbD67H7fJAtH5e8Zxrkc
To: /content/Alternative_Fuel_Stations_in_New_York.xlsx
100% 639k/639k [00:00<00:00, 84.2MB/s]


In [12]:
filename = 'Alternative_Fuel_Stations_in_New_York.xlsx'
ny_stations = pd.read_excel(filename)

## Exploratoring the dataframe

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
ny_stations.sample(2)

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Georeference
22,ELEC,32 Burling Lane,32 Burling Lane,NaN,New Rochelle,NY,10801.0,NaN,844-623-1001,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-73.78588 40.9129)
2242,ELEC,SUNY BUFFALO PARKER 6,185 Hayes Rd,NaN,Buffalo,NY,14214.0,NaN,888-758-4389,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-78.822603 42.951019)


In [5]:
ny_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3405 entries, 0 to 3404
Data columns (total 65 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Fuel Type Code                    3405 non-null   object        
 1   Station Name                      3405 non-null   object        
 2   Street Address                    3405 non-null   object        
 3   Intersection Directions           117 non-null    object        
 4   City                              3401 non-null   object        
 5   State                             3401 non-null   object        
 6   ZIP                               3401 non-null   float64       
 7   Plus4                             0 non-null      float64       
 8   Station Phone                     3388 non-null   object        
 9   Status Code                       3401 non-null   object        
 10  Expected Date                     0 non-null    

In [6]:
ny_stations.isna().mean()

Fuel Type Code                      0.000000
Station Name                        0.000000
Street Address                      0.000000
Intersection Directions             0.965639
City                                0.001175
State                               0.001175
ZIP                                 0.001175
Plus4                               1.000000
Station Phone                       0.004993
Status Code                         0.001175
Expected Date                       1.000000
Groups With Access Code             0.001175
Access Days Time                    0.245815
Cards Accepted                      0.849046
BD Blends                           0.999119
NG Fill Type Code                   0.997063
NG PSI                              0.997063
EV Level1 EVSE Num                  0.999706
EV Level2 EVSE Num                  0.150367
EV DC Fast Count                    0.949486
EV Other Info                       1.000000
EV Network                          0.105433
EV Network

### Verify the unique values per column

In [7]:
(ny_stations.dropna(axis=1, how='all')
            .nunique())

Fuel Type Code                         5
Station Name                        3328
Street Address                      2691
Intersection Directions              111
City                                 612
State                                  1
ZIP                                  732
Station Phone                        777
Status Code                            1
Groups With Access Code                6
Access Days Time                      79
Cards Accepted                        40
BD Blends                              2
NG Fill Type Code                      1
NG PSI                                 2
EV Level1 EVSE Num                     1
EV Level2 EVSE Num                    22
EV DC Fast Count                       9
EV Network                            18
EV Network Web                        17
Geocode Status                         3
Latitude                            3132
Longitude                           3131
Date Last Confirmed                   26
ID              

In [17]:
ny_stations_exp = (ny_stations
     .assign(
         Latitude = lambda dataset : dataset.Latitude.map(lambda lat: lat/(10**6)),
         Longitude = lambda dataset : dataset.Longitude.map(lambda lon: lon/(10**6))
   ))


fig = px.scatter_mapbox(ny_stations_exp,
                        lat='Latitude',
                        lon='Longitude',
                        zoom = 5,
                        hover_name = 'Station Name',
                        color_discrete_sequence = ['cyan'],
                        mapbox_style='carto-darkmatter')
fig.show()

## Cleaning the dataset

In [20]:
def fix_date_format(date):
    result = re.search(r'(\d\d)/(\d\d)/(\d\d\d\d)', str(date))
    if result != None:
      (month, day, year) = result.groups()
      date = f'{year}-{month}-{day}'
    return date

missing_values = {'EV_Network':'Not electric', 'EV_Connector_Types': 'Not electric'}

ny_stations = (ny_stations  
    .filter(['Fuel Type Code', 'Station Name', 'Street Adress','City',
             'ZIP','Station Code','EV Network','Latitude',
             'Longitude','Date Last Confirmed','Open Date',
             'EV Connector Types'])
    .rename(columns=lambda name: re.sub(' ','_',name))   
    .fillna(missing_values)
    .dropna()
    .assign(
           Date_Last_Confirmed = lambda dataset : dataset.Date_Last_Confirmed.map(fix_date_format),
           Open_Date = lambda dataset : dataset.Open_Date.map(fix_date_format)
    )
    .assign(
         Date_Last_Confirmed = lambda dataset : pd.to_datetime(dataset.Date_Last_Confirmed, format='%Y-%m-%d'),
         Open_Date = lambda dataset : pd.to_datetime(dataset.Open_Date, format='%Y-%m-%d'),
         Latitude = lambda dataset : dataset.Latitude.map(lambda lat: lat/(10**6)),
         Longitude = lambda dataset : dataset.Longitude.map(lambda lon: lon/(10**6)),
        ZIP = lambda dataset : dataset.ZIP.astype('int')
   )
   .query('Longitude > -80 and Longitude < -70')
)

<ipython-input-20-0baa8032c3d1>:19: FutureWarning:

Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])

<ipython-input-20-0baa8032c3d1>:20: FutureWarning:

Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])



In [9]:
ny_stations.isna().mean()

Fuel_Type_Code         0.0
Station_Name           0.0
City                   0.0
ZIP                    0.0
EV_Network             0.0
Latitude               0.0
Longitude              0.0
Date_Last_Confirmed    0.0
Open_Date              0.0
EV_Connector_Types     0.0
dtype: float64

## Visualización

In [ ]:
fig = px.scatter_mapbox(ny_stations,
                        lat='Latitude',
                        lon='Longitude',
                        zoom = 5,
                        hover_name = 'Station_Name',
                        hover_data = ['Fuel_Type_Code', 'EV_Connector_Types'],
                        color = 'Fuel_Type_Code',
                        mapbox_style='carto-darkmatter')

fig.show()

In [21]:
fig = px.scatter_mapbox(ny_stations,
                        lat='Latitude',
                        lon='Longitude',
                        zoom = 5,
                        hover_name = 'Station_Name',
                        hover_data = ['Fuel_Type_Code', 'EV_Network',],
                        color = 'EV_Network',
                        mapbox_style='carto-darkmatter')

fig.show()